## Homework

The goal of this homework is to get familiar with tools like MLflow for experiment tracking and 
model management.


## Q1. Install the package

To get started with MLflow you'll need to install the appropriate Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), 
and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [2]:
import mlflow

In [1]:
!mlflow --version

mlflow, version 2.3.2


## Download the data

Download the parquet files into data/raw folder

In [1]:
!(cd ./data/raw && curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet)
!(cd ./data/raw && curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet)
!(cd ./data/raw && curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1224k  100 1224k    0     0  1419k      0 --:--:-- --:--:-- --:--:-- 1417k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1394k  100 1394k    0     0  1739k      0 --:--:-- --:--:-- --:--:-- 1736k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1577k  100 1577k    0     0  2046k      0 --:--:-- --:--:-- --:--:-- 2043k


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the amount of tips for each trip. 

Download the data for January, February and March 2022 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2022 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

So what's the size of the saved `DictVectorizer` file?

* 54 kB
* 154 kB
* 54 MB
* 154 MB

In [4]:
!python -m homework.preprocess_data --raw_data_path ./data/raw --dest_path ./data/preprocess --dataset green

In [5]:
!du -sh ./data/preprocess/dv.pkl

152K	./data/preprocess/dv.pkl


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `max_depth` parameter:

* 4
* 6
* 8
* 10

In [ ]:
!python -m homework.train --data_path ./data/preprocess 

Answer: 10

## Launch the tracking server locally for MLflow

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

In case of MLflow, you need to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next three exercises that use the server.

Run the following command in your bash:

`mlflow server --backend-store-uri sqlite:///backend.db`

## Q4. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `optuna`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2022 data).

What's the best validation RMSE that you got?

* 1.85
* 2.15
* 2.45
* 2.85

* Run the hpo.py script.

In [ ]:
!python -m homework.hpo --data_path ./data/preprocess  --num_trials 10

In [43]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'sqlite:///backend.db'
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
experiment_id = 3

In [44]:
runs = client.search_runs(experiment_id, order_by=["metrics.rmse ASC"], max_results=1)
for run in runs:
    id,rmse = run.info.run_id, run.data.metrics['rmse']
    print(f'run id={id}, rmse={rmse:.2f}')


run id=d8aa33a905604295ab6472d4e2cd2aa8, rmse=2.45
